In [ ]:
!pip install -q -U transformers[torch]
!pip install -q -U datasets
!pip install -q -U evaluate
!pip install -q -U tokenizers
!pip install -q -U bitsandbytes
!pip install -q rouge_score
!pip install -q -U peft
!pip install -q -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import re
import random
import numpy as np
from scipy.special import softmax
import pprint

import bitsandbytes as bnb

import torch
import transformers
import evaluate
from datasets import Dataset, load_dataset

# For from-scratch T5 model
from transformers import T5TokenizerFast, T5Config, T5ForConditionalGeneration

# For pre-trained T5 model
from transformers import T5Tokenizer, T5ForConditionalGeneration  # this won't import twice, just noting here what's for each model

# For all T5 models
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, Trainer, TrainingArguments

# For BLEURT (to load a trained model for evaluation)
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM


# For style classifier model (also for evaluating the seq2seq model output)
from transformers import BertTokenizer, BertForSequenceClassification, BitsAndBytesConfig,pipeline
from transformers import TrainingArguments, Trainer

import pandas as pd
from tqdm import tqdm

# Local Data Loading Load Data

In [ ]:
# This cell will authenticate you and mount your Drive in the Colab.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Path to data save in Drive
train = 'FairytaleQA_train.csv'
valid = 'FairytaleQA_valid.csv'
test = 'FairytaleQA_test.csv'
path = 'drive/MyDrive/266_Danielle_Dylan_final_project/data/'
# path = 'drive/MyDrive/DataSci_266/266_Danielle_Dylan_final_project/data/'

In [ ]:
train_filepath = path+train
valid_filepath = path+valid
test_filepath = path+test

In [ ]:
# Save this because we'll need to tell the trainer how many examples we have
num_train_examples = 8_548

In [ ]:
train = load_dataset("csv",data_files=train_filepath,streaming=True)['train']
val = load_dataset("csv",data_files=valid_filepath,streaming=True)['train']
test = load_dataset("csv",data_files=test_filepath,streaming=True)['train']

In [ ]:
train

IterableDataset({
    features: Unknown,
    num_shards: 1
})

In [ ]:
from datasets import load_dataset, DatasetDict

# Combine into a single DatasetDict
ds = DatasetDict({
    "train": train,
    "test": test,
    "validation": val,
})

In [ ]:
train = ds['train'].shuffle()
val = ds['validation'].shuffle()
test = ds['test'].shuffle()

# Remote Data Loading

In [ ]:
# Initialize pipeline
model_id = "google/flan-t5-small"

In [ ]:
"""
Initialize the pipeline with bitsandbytes quantization
"""
# Configure bitsandbytes for 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


In [ ]:
VOCAB_SIZE = 15000
MAX_SEQUENCE_LENGTH = 512

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(model_id)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
# We prefix our tasks with "answer the question"
prefix = "Please answer this question: "
context = " Context: "

# Define the preprocessing function

def preprocess_function(data):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   data['full_input'] = [context + story + prefix + question for question,story in zip(data["question"],data['story_section'])]

   input_encoded = tokenizer.batch_encode_plus(data['full_input'],
                      max_length=MAX_SEQUENCE_LENGTH,
                      truncation=True,
                      padding='max_length',
                            return_tensors='pt')

   # The "labels" are the tokenized outputs:
   answers_encoded = tokenizer.batch_encode_plus(data["answer1"],
                      max_length=MAX_SEQUENCE_LENGTH,
                      truncation=True,
                      padding='max_length',
                      return_tensors='pt')

   return {'input_ids': input_encoded['input_ids'],
           'labels': answers_encoded['input_ids']}

In [ ]:
def create_seq2seq_training_args(batch_size, num_epochs):

    training_args = Seq2SeqTrainingArguments(
        "fairytale_QA_model",
        eval_strategy='epoch',
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        report_to='none'

    )

    return training_args

In [ ]:
def create_seq2seq_trainer(model, training_args, train_ds, val_ds):

    trainer = Seq2SeqTrainer(
        model,
        training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds
    )

    return trainer

In [ ]:
def create_training_args(batch_size, num_epochs):

    training_args = TrainingArguments(
        "fairytale_QA_model",
        eval_strategy='epoch',
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        report_to='none'

    )

    return training_args

In [ ]:
def create_trainer(model, training_args, train_ds, val_ds):

    trainer = Trainer(
        model,
        training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds
    )

    return trainer

# Train Individual T-5 Lora Models on Implicit vs Explicit

In [ ]:
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
import torch
from transformers import AutoTokenizer

## Explicit

In [ ]:
# Save this because we'll need to tell the trainer how many examples we have
num_train_examples_ex = 6_382

In [ ]:
# Configure bitsandbytes for 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
rouge = evaluate.load('rouge')

def compute_metrics_rouge(p):
  predictions, labels = p
  return rouge.compute(predictions=predictions, references=labels)


In [ ]:
# Load the BLEU metric and the trained BLEURT model for semantic similarity scoring

bleu = evaluate.load("bleu")

bleurt_tokenizer = AutoTokenizer.from_pretrained("Elron/bleurt-base-512")
bleurt_model = AutoModelForSequenceClassification.from_pretrained("Elron/bleurt-base-512")

tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
model_checkpoint = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# lr = 1e-5
# batch_size = 16
# num_epochs = 3

lr = 1e-3
batch_size = 16
num_epochs = 5
# set LORA specific hyperparameters
r = 8
lora_alpha = 32
lora_dropout = 0.1

In [ ]:
# # set LORA specific hyperparameters
# r = 8
# lora_alpha = 32
# lora_dropout = 0.1

In [ ]:
model_ex = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint,quantization_config=quantization_config)

# for name, param in model.named_parameters():
#     param.requires_grad = False

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout
)

In [ ]:
model_ex = get_peft_model(model_ex, peft_config)
model_ex.print_trainable_parameters()

trainable params: 344,064 || all params: 77,305,216 || trainable%: 0.4451


In [ ]:
_= model_ex.to("cuda")

In [ ]:
# Map the preprocessing function across our dataset
train_ex = train.filter(lambda x: x['ex_or_im'] == 'explicit')
val_ex = val.filter(lambda x: x['ex_or_im'] == 'explicit')
test_ex = test.filter(lambda x: x['ex_or_im'] == 'explicit')

train_tokenized_ex = train_ex.map(preprocess_function, batched=True)
val_tokenized_ex = val_ex.map(preprocess_function, batched=True)
test_tokenized_ex = test_ex.map(preprocess_function, batched=True)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="t5-small-lora-text-generation",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    max_steps=int(num_epochs * num_train_examples_ex / batch_size),
    report_to='none'
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model_ex,
    args=training_args,
    train_dataset=train_tokenized_ex,
    eval_dataset=val_tokenized_ex,
    #compute_metrics=compute_metrics_rouge,
)

trainer.train()

No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
0,4.393400,4.324219
1,4.409400,4.343750
2,4.410900,4.332031
3,4.417200,4.351562
4,4.432400,4.355469


TrainOutput(global_step=1994, training_loss=4.687919226429288, metrics={'train_runtime': 2245.4559, 'train_samples_per_second': 14.208, 'train_steps_per_second': 0.888, 'total_flos': 5962876481175552.0, 'train_loss': 4.687919226429288, 'epoch': 4.199598796389168})

In [ ]:
model_ex_checkpoint_filepath = 'drive/MyDrive/266_Danielle_Dylan_final_project/checkpoints/t5_model_ex_smaller_lr'
model_ex.save_pretrained(model_ex_checkpoint_filepath, from_pt=True)

In [ ]:
# Run this line only if you need to reload the model you trained earlier
model_ex = AutoModelForSeq2SeqLM.from_pretrained(model_ex_checkpoint_filepath)

In [ ]:
pipe = pipeline('text2text-generation',model=model_ex,tokenizer =tokenizer)

Device set to use cuda:0


In [ ]:
%%time

# Store results for aggregate scoring

def process_dataset(dataset):
  results = []
  bleurt_scores = []
  for idx, sample in enumerate(tqdm(dataset)):

      # if idx >= 10:  # Stop after processing 10 samples
      #   return pd.DataFrame(results)
      #   break

      prefix = "Please answer this question: "
      context = " Context: "

      question = context + sample['story_section'] + prefix + sample['question']

      # Generate summary via the pipeline
      outputs = pipe(
                          question,
                          max_new_tokens=500,
                          truncation=True
      )

      answer = outputs[0]["generated_text"]

      if answer.isspace():
        print("\nANSWER IS SPACE for question:",sample['question'],"\nReference:", sample['answer1'])
        print("\nanswer:",[answer])
        answer = 'blank'

      # print("\nbegin answer:", answer, ":end answer")
      # print("\n",[answer],"****")

      # Evaluation
      predictions = [answer]
      reference1 = [sample['answer1']]
      reference2 = [sample['answer2']]

      # Calculate ROUGE scores
      rouge_scores1 = rouge.compute(predictions=predictions, references=reference1)
      if reference1 != reference2:
        rouge_scores2 = rouge.compute(predictions=predictions, references=reference2)
      else:
        rouge_scores2 = rouge_scores1
      rouge_scores1 = {k+'_a1': v for k, v in rouge_scores1.items()}
      rouge_scores2 = {k+'_a2': v for k, v in rouge_scores2.items()}


      # Calculate BLEU scores
      bleu_scores1 = bleu.compute(predictions=predictions, references=reference1,smooth=True)
      if reference1 != reference2:
        bleu_scores2 = bleu.compute(predictions=predictions, references=reference2,smooth=True)
      else:
        bleu_scores2 = bleu_scores1
      bleu_scores1 = {k+'_a1': v for k, v in bleu_scores1.items()}
      bleu_scores2 = {k+'_a2': v for k, v in bleu_scores2.items()}

      # Calculate BLEURT scores
      with torch.no_grad():
        bleurt_score_a1 = bleurt_model(**bleurt_tokenizer(answer,
                                                sample['answer1'],
                                                truncation=True,
                                                max_length=512,
                                                padding='max_length',
                                                return_tensors='pt'))[0].squeeze().numpy()
        if reference1 != reference2:
          bleurt_score_a2 = bleurt_model(**bleurt_tokenizer(answer,
                                                sample['answer2'],
                                                truncation=True,
                                                max_length=512,
                                                padding='max_length',
                                                return_tensors='pt'))[0].squeeze().numpy()
        else:
          bleurt_score_a2 = bleurt_score_a1


      # Store results
      results.append({
          'id': idx,
          'question': sample['question'],
          'ex_or_im': sample['ex_or_im'],
          'story_section': sample['story_section'][:500],  # Store truncated text for readability
          'reference_answer1': sample['answer1'],
          'reference_answer2': sample['answer2'],
          'generated_answer': answer,
          **rouge_scores1,
          **rouge_scores2,
          **bleu_scores1,
          **bleu_scores2,
          'bleurt_score_a1': bleurt_score_a1,
          'bleurt_score_a2': bleurt_score_a2,
      })

      # Print progress update every 10 samples
      if (idx + 1) % 100 == 0:
          print(f"\nProcessed {idx + 1} samples; Results length: {len(results)}")
          print(f"Latest Answer1 ROUGE-1: {rouge_scores1['rouge1_a1']:.4f}")

  return pd.DataFrame(results)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.11 µs


In [ ]:
test_ex_ds = Dataset.from_list(list(test_ex))
test_ex_ds

Dataset({
    features: ['story_name', 'story_section', 'question', 'answer1', 'answer2', 'local_or_sum', 'attribute', 'ex_or_im', 'ex_or_im2'],
    num_rows: 754
})

In [ ]:
test_results_ex_df = process_dataset(test_ex_ds)

  0%|          | 0/754 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
 13%|█▎        | 100/754 [04:16<26:16,  2.41s/it]


Processed 100 samples; Results length: 100
Latest Answer1 ROUGE-1: 0.0000


 27%|██▋       | 200/754 [08:35<25:35,  2.77s/it]


Processed 200 samples; Results length: 200
Latest Answer1 ROUGE-1: 0.5000


 40%|███▉      | 300/754 [12:41<15:03,  1.99s/it]

...

 80%|███████▉  | 600/754 [24:58<06:42,  2.61s/it]


Processed 600 samples; Results length: 600
Latest Answer1 ROUGE-1: 0.1818


 93%|█████████▎| 700/754 [28:55<02:22,  2.64s/it]


Processed 700 samples; Results length: 700
Latest Answer1 ROUGE-1: 0.8511


100%|██████████| 754/754 [31:10<00:00,  2.48s/it]


### Explicit Generation Experimentations

In [ ]:
# def generate_output(model, tokenizer, data, **kwargs):

#     all_outputs = []
#     prefix = "Please answer this question: "
#     context = " Context: "
#     input_sentences = [context + sample['story_section'] + prefix + sample['question'] for sample in data]

#     for i in range(int(len(input_sentences)) + 1):
#         if i >= len(input_sentences):
#             break

#         inputs_encoded = tokenizer(input_sentences[i],truncation=True, padding=True, return_tensors='pt')
#         output_ids = model.cuda().generate(input_ids=inputs_encoded['input_ids'].cuda(), **kwargs)
#         generated_sentences = tokenizer.batch_decode(output_ids,
#                                                      skip_special_tokens=True,
#                                                      clean_up_tokenization_spaces=False)
#         all_outputs.extend(generated_sentences)

#     return all_outputs

In [ ]:
# def calculate_eval_metrics(data, model, tokenizer, prefix="", **kwargs):

#     # Translate original texts
#     answers = generate_output(model, tokenizer, data, **kwargs)

#     # Calculate ROUGE scores
#     rouge_results = rouge.compute(predictions=answers, references=data['answer1'])
#     print('ROUGE: ', rouge_results)

#     # Calculate BLEURT scores
#     bleurt_scores = []
#     for i in range(int(len(answers)) + 1):
#         if i >= len(answers):
#             break

#         with torch.no_grad():
#             scores = bleurt_model(**bleurt_tokenizer(data['answer1'][i],
#                                                      answers[i],
#                                                      truncation=True,
#                                                      max_length=MAX_SEQUENCE_LENGTH,
#                                                      padding='max_length',
#                                                      return_tensors='pt'))[0].squeeze().numpy()
#             if scores.shape:
#                 bleurt_scores.extend(scores)
#             else:  # Happens when there was only one example in the last batch
#                 bleurt_scores.append(float(scores))

#     print('BLEURT: ', np.mean(bleurt_scores))

#     return answers

In [ ]:
# """
# Playing with the decoder .generate() arguments like num_beams or top_p, etc.
# """

# generate_kwargs = {

#     'num_beams': 4,
#     'do_sample':True,
#     'top_k':100,
#     'top_p':1,
#     'temperature':0.2,
#     'no_repeat_ngram_size':2
# }

# val_ex_answers = calculate_eval_metrics(
#     val_small.select(range(100)),
#     model_ex,
#     tokenizer,
#     **generate_kwargs
# )

In [ ]:
# # Print out a sample of outputs to manually review
# for i in range(10):
#     sample_i = random.choice(range(len(val_ex_answers)))
#     print('Referenced answer:  ', val_small['answer1'][i])
#     print('Generated answer:   ', val_ex_answers[i])
#     print()

In [ ]:
def clear_gpu_memory():
    import gc
    import torch

    gc.collect()
    torch.cuda.empty_cache()
    print("Cleared GPU memory.")

# Usage
del model_ex, tokenizer, trainer
clear_gpu_memory()


Cleared GPU memory.


## Implicit

In [ ]:
num_train_examples_im = 2_166

In [ ]:
# Configure bitsandbytes for 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
model_checkpoint = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

lr = 1e-3
batch_size = 16
num_epochs = 5

# set LORA specific hyperparameters
r = 8
lora_alpha = 32
lora_dropout = 0.1

In [ ]:
model_im = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint,quantization_config=quantization_config)

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout
)

In [ ]:
model_im = get_peft_model(model_im, peft_config)
model_im.print_trainable_parameters()

trainable params: 344,064 || all params: 77,305,216 || trainable%: 0.4451


In [ ]:
_= model_im.to("cuda")

In [ ]:
# Map the preprocessing function across our dataset
train_im = train.filter(lambda x: x['ex_or_im'] == 'implicit')
val_im = val.filter(lambda x: x['ex_or_im'] == 'implicit')
test_im = test.filter(lambda x: x['ex_or_im'] == 'implicit')

train_tokenized_im = train_im.map(preprocess_function, batched=True)
val_tokenized_im = val_im.map(preprocess_function, batched=True)
test_tokenized_im = test_im.map(preprocess_function, batched=True)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="t5-small-lora-text-generation",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    max_steps=int(num_epochs * num_train_examples_im / batch_size),
    report_to='none'
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model_im,
    args=training_args,
    train_dataset=train_tokenized_im,
    eval_dataset=val_tokenized_im,
    #compute_metrics=compute_metrics_rouge,
)

trainer.train()

No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
0,4.540600,4.453125
1,4.489800,4.429688
2,4.490200,4.437500
3,4.471500,4.429688
4,4.467200,4.421875


TrainOutput(global_step=676, training_loss=5.302589913091716, metrics={'train_runtime': 765.6227, 'train_samples_per_second': 14.127, 'train_steps_per_second': 0.883, 'total_flos': 2014545929306112.0, 'train_loss': 5.302589913091716, 'epoch': 4.195266272189349})

In [ ]:
model_im_checkpoint_filepath = 'drive/MyDrive/266_Danielle_Dylan_final_project/checkpoints/t5_model_im_smaller_lr'
model_im.save_pretrained(model_im_checkpoint_filepath, from_pt=True)

In [ ]:

# Run this line only if you need to reload the model you trained earlier
model_im = AutoModelForSeq2SeqLM.from_pretrained(model_im_checkpoint_filepath)

In [ ]:
test_im_ds = Dataset.from_list(list(test_im))
test_results_im_df = process_dataset(test_im_ds)

  0%|          | 0/253 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
 40%|███▉      | 100/253 [04:52<07:40,  3.01s/it]


Processed 100 samples; Results length: 100
Latest Answer1 ROUGE-1: 0.1739


 79%|███████▉  | 200/253 [09:43<02:49,  3.20s/it]


Processed 200 samples; Results length: 200
Latest Answer1 ROUGE-1: 0.0294


100%|██████████| 253/253 [12:14<00:00,  2.90s/it]


In [ ]:
test_both_outputs = pd.concat([test_results_ex_df,test_results_im_df])
test_both_outputs.shape

(1007, 29)

In [ ]:
# test_both_outputs.to_csv('drive/MyDrive/266_Danielle_Dylan_final_project/answer_outputs/t5_trained_specialized.csv',index=False)
test_both_outputs.to_csv('drive/MyDrive/266_Danielle_Dylan_final_project/answer_outputs/t5_trained_specialized_smaller_lr.csv',index=False)

### Implicit Generation Experimentation

In [ ]:
# def generate_output(model, tokenizer, data, **kwargs):

#     all_outputs = []
#     prefix = "Please answer this question: "
#     context = " Context: "
#     input_sentences = [context + sample['story_section'] + prefix + sample['question'] for sample in data]

#     for i in range(int(len(input_sentences)) + 1):
#         if i >= len(input_sentences):
#             break

#         inputs_encoded = tokenizer(input_sentences[i],truncation=True, padding=True, return_tensors='pt')
#         output_ids = model.cuda().generate(input_ids=inputs_encoded['input_ids'].cuda(), **kwargs)
#         generated_sentences = tokenizer.batch_decode(output_ids,
#                                                      skip_special_tokens=True,
#                                                      clean_up_tokenization_spaces=False)
#         all_outputs.extend(generated_sentences)

#     return all_outputs

In [ ]:
# def calculate_eval_metrics(data, model, tokenizer, prefix="", **kwargs):

#     # Translate original texts
#     answers = generate_output(model, tokenizer, data, **kwargs)

#     # Calculate ROUGE scores
#     rouge_results = rouge.compute(predictions=answers, references=data['answer1'])
#     print('ROUGE: ', rouge_results)

#     # Calculate BLEURT scores
#     bleurt_scores = []
#     for i in range(int(len(answers)) + 1):
#         if i >= len(answers):
#             break

#         with torch.no_grad():
#             scores = bleurt_model(**bleurt_tokenizer(data['answer1'][i],
#                                                      answers[i],
#                                                      truncation=True,
#                                                      max_length=MAX_SEQUENCE_LENGTH,
#                                                      padding='max_length',
#                                                      return_tensors='pt'))[0].squeeze().numpy()
#             if scores.shape:
#                 bleurt_scores.extend(scores)
#             else:  # Happens when there was only one example in the last batch
#                 bleurt_scores.append(float(scores))

#     print('BLEURT: ', np.mean(bleurt_scores))

#     return answers

In [ ]:
# """
# Playing with the decoder .generate() arguments like num_beams or top_p, etc.
# """

# generate_kwargs = {

#     'num_beams': 4,
#     'do_sample':True,
#     'top_k':100,
#     'top_p':1,
#     'temperature':0.2,
#     'no_repeat_ngram_size':2
# }

# val_answers_im = calculate_eval_metrics(
#     val_small.select(range(100)),
#     model_im,
#     tokenizer,
#     batch_size,
#     **generate_kwargs
# )

In [ ]:
# # Print out a sample of outputs to manually review
# for i in range(10):
#     sample_i = random.choice(range(len(val_answers_im)))
#     print('Referenced answer:  ', val_small['answer1'][i])
#     print('Generated answer:   ', val_answers_im[i])
#     print()

In [ ]:
def clear_gpu_memory():
    import gc
    import torch

    gc.collect()
    torch.cuda.empty_cache()
    print("Cleared GPU memory.")

# Usage
del model_im, tokenizer, trainer
clear_gpu_memory()
